In [2]:
pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00


In [3]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np
import time

class CombinedModel:
    def __init__(self):
        self.screener = Overview()

    def get_sector_data(self, sector):
        filters_dict = {
            'Exchange': 'NASDAQ',
            'Market Cap.': '+Mid (over $2bln)',
            'Average Volume': 'Over 500K',
            'Price': 'Over $10',
            'Sector': sector
        }
        self.screener.set_filter(filters_dict=filters_dict)
        return self.screener.screener_view()

    def calculate_xsmom(self, ticker, universe_returns):
        stock = yf.Ticker(ticker)
        hist = stock.history(period="6mo")
        if hist.empty:
            return 0

        stock_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
        mean_return = universe_returns['Returns'].mean()
        std_return = universe_returns['Returns'].std()

        return (stock_return - mean_return) / std_return

    def get_stock_fundamentals(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1y")

            returns = hist['Close'].pct_change()
            growth_rate = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
            volatility = returns.std() * np.sqrt(252)

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0),
                'GrowthRate': growth_rate,
                'Volatility': volatility
            }
        except Exception as e:
            print(f"Error getting data for {ticker}: {str(e)}")
            return None

    def analyze_sector(self, sector):
        # Get sector stocks
        df = self.get_sector_data(sector)
        if df.empty:
            return pd.DataFrame()

        # Calculate universe returns for XSMOM
        universe_returns = []
        for ticker in df['Ticker']:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="6mo")
                if not hist.empty:
                    returns = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
                    universe_returns.append({
                        'Ticker': ticker,
                        'Returns': returns
                    })
                time.sleep(0.1)
            except:
                continue

        universe_returns_df = pd.DataFrame(universe_returns)

        # Get fundamentals and calculate scores
        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_stock_fundamentals(ticker)
            if data:
                xsmom = self.calculate_xsmom(ticker, universe_returns_df)
                data['XSMOM_Score'] = xsmom
                detailed_data.append(data)

        if not detailed_data:
            return pd.DataFrame()

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        # Calculate Quality Score
        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity', 'GrowthRate', 'Volatility']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        growth = df['GrowthRate']
        safety = -(df[['Beta', 'DebtToEquity', 'Volatility']].mean(axis=1))

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Growth_Score'] = (growth - growth.mean()) / growth.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()

        df['Quality_Score'] = (df['Profitability_Score'] + df['Growth_Score'] + df['Safety_Score']) / 3

        # Combined Score
        df['Combined_Score'] = (df['Quality_Score'] + df['XSMOM_Score']) / 2

        return df.sort_values('Combined_Score', ascending=False)

    def get_portfolio(self, n_stocks=5):
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            sector_data = self.analyze_sector(sector)
            if not sector_data.empty:
                all_data.append(sector_data)

        if not all_data:
            raise ValueError("No data available for analysis")

        df = pd.concat(all_data, ignore_index=True)
        df = df.sort_values('Combined_Score', ascending=False)

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'spread': high_quality['Combined_Score'].mean() - low_quality['Combined_Score'].mean()
        }

# Usage example
model = CombinedModel()
portfolio = model.get_portfolio()
print("\nLong Portfolio (High Quality + Momentum):")
print(portfolio['long'])
print("\nShort Portfolio (Low Quality + Momentum):")
print(portfolio['short'])
print("\nSpread:", portfolio['spread'])

ERROR:yfinance:NBIS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:TTAN: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:NBIS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Error getting data for NBIS: single positional indexer is out-of-bounds


ERROR:yfinance:OS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']


Error getting data for OS: single positional indexer is out-of-bounds


ERROR:yfinance:TTAN: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']


...

Error getting data for PPC: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SFM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SFM&crumb=Edge%3A+Too+Many+Requests


Error getting data for SFM: Expecting value: line 1 column 1 (char 0)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SMPL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SMPL&crumb=Edge%3A+Too+Many+Requests


Error getting data for SMPL: Expecting value: line 1 column 1 (char 0)

Long Portfolio (High Quality + Momentum):
    Ticker                             Company      Sector  Combined_Score  \
0      APP                       Applovin Corp  Technology        2.997189   
1       KC     Kingsoft Cloud Holdings Ltd ADR  Technology        2.666265   
140   GGAL        Grupo Financiero Galicia ADR   Financial        2.588492   
2     PLTR           Palantir Technologies Inc  Technology        1.847653   
3     CRDO  Credo Technology Group Holding Ltd  Technology        1.604569   

     Quality_Score  XSMOM_Score  
0         0.947284     5.047094  
1         0.551682     4.780848  
140       1.190477     3.986506  
2         1.338468     2.356837  
3         0.901253     2.307884  

Short Portfolio (Low Quality + Momentum):
    Ticker            Company      Sector  Combined_Score  Quality_Score  \
191   MARA  MARA Holdings Inc   Financial       -0.974508      -1.092267   
138    RUN        

**QJM + XSMOM +HRP**




In [5]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
import time

class CombinedModel:
    def __init__(self):
        self.screener = Overview()

    def get_sector_data(self, sector):
        filters_dict = {
            'Exchange': 'NASDAQ',
            'Market Cap.': '+Mid (over $2bln)',
            'Average Volume': 'Over 500K',
            'Price': 'Over $10',
            'Sector': sector
        }
        self.screener.set_filter(filters_dict=filters_dict)
        return self.screener.screener_view()

    def calculate_xsmom(self, ticker, universe_returns):
        stock = yf.Ticker(ticker)
        hist = stock.history(period="6mo")
        if hist.empty:
            return 0

        stock_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
        mean_return = universe_returns['Returns'].mean()
        std_return = universe_returns['Returns'].std()

        return (stock_return - mean_return) / std_return

    def get_stock_fundamentals(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1y")

            returns = hist['Close'].pct_change()
            growth_rate = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
            volatility = returns.std() * np.sqrt(252)

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0),
                'GrowthRate': growth_rate,
                'Volatility': volatility
            }
        except Exception as e:
            print(f"Error getting data for {ticker}: {str(e)}")
            return None

    def analyze_sector(self, sector):
        df = self.get_sector_data(sector)
        if df.empty:
            return pd.DataFrame()

        universe_returns = []
        for ticker in df['Ticker']:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="6mo")
                if not hist.empty:
                    returns = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
                    universe_returns.append({
                        'Ticker': ticker,
                        'Returns': returns
                    })
                time.sleep(0.1)
            except:
                continue

        universe_returns_df = pd.DataFrame(universe_returns)

        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_stock_fundamentals(ticker)
            if data:
                xsmom = self.calculate_xsmom(ticker, universe_returns_df)
                data['XSMOM_Score'] = xsmom
                detailed_data.append(data)

        if not detailed_data:
            return pd.DataFrame()

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity', 'GrowthRate', 'Volatility']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        growth = df['GrowthRate']
        safety = -(df[['Beta', 'DebtToEquity', 'Volatility']].mean(axis=1))

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Growth_Score'] = (growth - growth.mean()) / growth.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()

        df['Quality_Score'] = (df['Profitability_Score'] + df['Growth_Score'] + df['Safety_Score']) / 3
        df['Combined_Score'] = (df['Quality_Score'] + df['XSMOM_Score']) / 2

        return df.sort_values('Combined_Score', ascending=False)

    def get_portfolio(self, n_stocks=5):
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            sector_data = self.analyze_sector(sector)
            if not sector_data.empty:
                all_data.append(sector_data)

        if not all_data:
            raise ValueError("No data available for analysis")

        df = pd.concat(all_data, ignore_index=True)
        df = df.sort_values('Combined_Score', ascending=False)

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'spread': high_quality['Combined_Score'].mean() - low_quality['Combined_Score'].mean()
        }

class CombinedModelHRP(CombinedModel):
    def __init__(self):
        super().__init__()

    def get_stock_returns(self, tickers, period='1y'):
        returns_data = {}
        for ticker in tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period=period)
                if not hist.empty:
                    returns = hist['Close'].pct_change().dropna()
                    returns_data[ticker] = returns
                time.sleep(0.1)
            except Exception as e:
                print(f"Error getting returns for {ticker}: {str(e)}")
        return pd.DataFrame(returns_data)

    def get_quasi_diag(self, link):
        num_items = link[-1, 3]
        cursors = [link[-1, 0], link[-1, 1]]
        sorted_items = []

        while len(sorted_items) < num_items:
            for cursor in cursors:
                if cursor < num_items:
                    sorted_items.append(int(cursor))
                    cursors.remove(cursor)
                else:
                    i = int(cursor - num_items)
                    cursors.remove(cursor)
                    cursors.extend([link[i, 0], link[i, 1]])

        return sorted_items

    def get_cluster_var(self, cov, cluster_indices):
        cluster_cov = cov.iloc[cluster_indices, cluster_indices]
        inv_diag = 1 / np.diag(cluster_cov)
        weights = inv_diag / inv_diag.sum()
        cluster_var = np.dot(np.dot(weights, cluster_cov), weights)
        return cluster_var, weights

    def optimize_portfolio(self, returns):
        if returns.empty:
            return pd.Series()

        corr = returns.corr()
        dist = np.sqrt(0.5 * (1 - corr))

        condensed_dist = squareform(dist)
        link = sch.linkage(condensed_dist, method='single')

        sorted_indices = self.get_quasi_diag(link)
        sorted_assets = returns.columns[sorted_indices].tolist()

        cov = returns[sorted_assets].cov()
        weights = pd.Series(1, index=sorted_assets)

        def recursive_bisection(indices, level=0):
            if len(indices) <= 1:
                return

            mid = len(indices) // 2
            left_indices = indices[:mid]
            right_indices = indices[mid:]

            left_var, left_weights = self.get_cluster_var(cov, left_indices)
            right_var, right_weights = self.get_cluster_var(cov, right_indices)

            alpha = 1 - (left_var / (left_var + right_var))

            for idx in left_indices:
                weights[sorted_assets[idx]] *= alpha
            for idx in right_indices:
                weights[sorted_assets[idx]] *= (1 - alpha)

            recursive_bisection(left_indices, level + 1)
            recursive_bisection(right_indices, level + 1)

        initial_indices = list(range(len(sorted_assets)))
        recursive_bisection(initial_indices)

        return weights

    def get_portfolio(self, n_stocks=5):
        base_portfolio = super().get_portfolio(n_stocks)

        long_tickers = base_portfolio['long']['Ticker'].tolist()
        returns = self.get_stock_returns(long_tickers)

        if not returns.empty:
            hrp_weights = self.optimize_portfolio(returns)
            hrp_weights = hrp_weights / hrp_weights.sum()
            base_portfolio['long']['HRP_Weight'] = base_portfolio['long']['Ticker'].map(hrp_weights)
        else:
            base_portfolio['long']['HRP_Weight'] = 1.0 / len(long_tickers)

        return base_portfolio

# Usage
model = CombinedModelHRP()
portfolio = model.get_portfolio()
print("\nLong Portfolio (High Quality + Momentum) with HRP Weights:")
print(portfolio['long'])
print("\nShort Portfolio (Low Quality + Momentum):")
print(portfolio['short'])
print("\nSpread:", portfolio['spread'])

ERROR:yfinance:NBIS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:TTAN: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:NBIS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Error getting data for NBIS: single positional indexer is out-of-bounds


ERROR:yfinance:OS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']


Error getting data for OS: single positional indexer is out-of-bounds


ERROR:yfinance:TTAN: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']


...

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZION?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZION&crumb=2FB6Ho5bejq
ERROR:yfinance:$ZION: possibly delisted; no price data found  (period=1y)


Error getting data for ZION: single positional indexer is out-of-bounds


/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)



Long Portfolio (High Quality + Momentum) with HRP Weights:
    Ticker                          Company      Sector  Combined_Score  \
172   SRRK        Scholar Rock Holding Corp  Healthcare        3.917468   
0      APP                    Applovin Corp  Technology        2.996741   
140   GGAL     Grupo Financiero Galicia ADR   Financial        2.682144   
1       KC  Kingsoft Cloud Holdings Ltd ADR  Technology        2.665849   
173    WGS             GeneDx Holdings Corp  Healthcare        2.020108   

     Quality_Score  XSMOM_Score  HRP_Weight  
172       0.787948     7.046987    0.012025  
0         0.947231     5.046251    0.205837  
140       1.377782     3.986507    0.569322  
1         0.551636     4.780063    0.110105  
173       0.947473     3.092743    0.102710  

Short Portfolio (Low Quality + Momentum):
    Ticker               Company              Sector  Combined_Score  \
298   CELH  Celsius Holdings Inc  Consumer Defensive       -1.086932   
138    RUN            Sunr

<ipython-input-5-e4c238cac011>:208: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5813469813484817' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights[sorted_assets[idx]] *= alpha


**QMJ + XSMOM + HRP + Turnover + Riesgos + Correlaciones**

In [7]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
import time

class CombinedModel:
    def __init__(self):
        self.screener = Overview()
        self._previous_weights = None

    def get_sector_data(self, sector):
        filters_dict = {
            'Exchange': 'NASDAQ',
            'Market Cap.': '+Mid (over $2bln)',
            'Average Volume': 'Over 500K',
            'Price': 'Over $10',
            'Sector': sector
        }
        self.screener.set_filter(filters_dict=filters_dict)
        return self.screener.screener_view()

    def calculate_xsmom(self, ticker, universe_returns):
        stock = yf.Ticker(ticker)
        hist = stock.history(period="6mo")
        if hist.empty:
            return 0

        stock_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
        mean_return = universe_returns['Returns'].mean()
        std_return = universe_returns['Returns'].std()

        return (stock_return - mean_return) / std_return

    def sector_momentum_score(self, sector_returns):
        return (sector_returns - sector_returns.mean()) / sector_returns.std()

    def get_stock_fundamentals(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1y")

            returns = hist['Close'].pct_change()
            growth_rate = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
            volatility = returns.std() * np.sqrt(252)

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0),
                'GrowthRate': growth_rate,
                'Volatility': volatility
            }
        except Exception as e:
            print(f"Error getting data for {ticker}: {str(e)}")
            return None

    def analyze_sector(self, sector):
        df = self.get_sector_data(sector)
        if df.empty:
            return pd.DataFrame()

        universe_returns = []
        for ticker in df['Ticker']:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="6mo")
                if not hist.empty:
                    returns = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
                    universe_returns.append({
                        'Ticker': ticker,
                        'Returns': returns
                    })
                time.sleep(0.1)
            except:
                continue

        universe_returns_df = pd.DataFrame(universe_returns)

        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_stock_fundamentals(ticker)
            if data:
                xsmom = self.calculate_xsmom(ticker, universe_returns_df)
                data['XSMOM_Score'] = xsmom
                detailed_data.append(data)

        if not detailed_data:
            return pd.DataFrame()

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity', 'GrowthRate', 'Volatility']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        growth = df['GrowthRate']
        safety = -(df[['Beta', 'DebtToEquity', 'Volatility']].mean(axis=1))

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Growth_Score'] = (growth - growth.mean()) / growth.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()

        df['Quality_Score'] = (df['Profitability_Score'] + df['Growth_Score'] + df['Safety_Score']) / 3
        df['Combined_Score'] = (df['Quality_Score'] + df['XSMOM_Score']) / 2

        return df.sort_values('Combined_Score', ascending=False)

    def get_portfolio(self, n_stocks=5):
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            sector_data = self.analyze_sector(sector)
            if not sector_data.empty:
                all_data.append(sector_data)

        if not all_data:
            raise ValueError("No data available for analysis")

        df = pd.concat(all_data, ignore_index=True)
        df = df.sort_values('Combined_Score', ascending=False)

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        if self._previous_weights is None:
            self._previous_weights = pd.Series(1.0/n_stocks, index=high_quality['Ticker'])

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'spread': high_quality['Combined_Score'].mean() - low_quality['Combined_Score'].mean()
        }

    def analyze_factor_correlation(self):
        factors = ['Quality_Score', 'XSMOM_Score', 'Sector_Momentum']
        return pd.DataFrame([self.quality_scores, self.momentum_scores, self.sector_scores],
                          index=factors).corr()

    def calculate_portfolio_costs(self, old_weights, new_weights, transaction_cost=2.0):
        weight_changes = abs(new_weights - old_weights)
        turnover = weight_changes.sum() / 2
        num_trades = len(weight_changes[weight_changes > 0.0001])
        total_costs = num_trades * transaction_cost
        portfolio_value = 100000
        costs_impact_bps = (total_costs / portfolio_value) * 10000

        return {
            'turnover_ratio': turnover,
            'number_of_trades': num_trades,
            'total_costs_usd': total_costs,
            'costs_impact_bps': costs_impact_bps
        }

    def calculate_risk_metrics(self, returns, confidence_level=0.95):
        if self._previous_weights is None:
            return None

        portfolio_returns = returns.dot(self._previous_weights)
        var = -np.percentile(portfolio_returns, (1 - confidence_level) * 100)
        cvar = -portfolio_returns[portfolio_returns <= -var].mean()

        var_limit = 0.02
        cvar_limit = 0.03

        alerts = []
        if var > var_limit:
            alerts.append({
                'level': 'WARNING',
                'message': f'VaR ({var:.2%}) exceeds limit of {var_limit:.2%}',
                'suggestion': 'Consider reducing position sizes or adding defensive stocks'
            })

        if cvar > cvar_limit:
            alerts.append({
                'level': 'CRITICAL',
                'message': f'CVaR ({cvar:.2%}) exceeds limit of {cvar_limit:.2%}',
                'suggestion': 'Immediate portfolio rebalancing recommended. Consider hedging positions'
            })

        marginal_var = {}
        for ticker in returns.columns:
            position_returns = returns[ticker]
            contribution = position_returns[portfolio_returns <= -var].mean() * self._previous_weights[ticker]
            marginal_var[ticker] = contribution

        return {
            'VaR': var,
            'CVaR': cvar,
            'risk_alerts': alerts,
            'risk_contributions': marginal_var
        }

class CombinedModelHRP(CombinedModel):
    def get_stock_returns(self, tickers, period='1y'):
        returns_data = {}
        for ticker in tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period=period)
                if not hist.empty:
                    returns = hist['Close'].pct_change().dropna()
                    returns_data[ticker] = returns
                time.sleep(0.1)
            except Exception as e:
                print(f"Error getting returns for {ticker}: {str(e)}")
        return pd.DataFrame(returns_data)

    def get_quasi_diag(self, link):
        num_items = link[-1, 3]
        cursors = [link[-1, 0], link[-1, 1]]
        sorted_items = []

        while len(sorted_items) < num_items:
            for cursor in cursors:
                if cursor < num_items:
                    sorted_items.append(int(cursor))
                    cursors.remove(cursor)
                else:
                    i = int(cursor - num_items)
                    cursors.remove(cursor)
                    cursors.extend([link[i, 0], link[i, 1]])

        return sorted_items

    def get_cluster_var(self, cov, cluster_indices):
        cluster_cov = cov.iloc[cluster_indices, cluster_indices]
        inv_diag = 1 / np.diag(cluster_cov)
        weights = inv_diag / inv_diag.sum()
        cluster_var = np.dot(np.dot(weights, cluster_cov), weights)
        return cluster_var, weights

    def optimize_portfolio(self, returns):
        if returns.empty:
            return pd.Series()

        corr = returns.corr()
        dist = np.sqrt(0.5 * (1 - corr))

        condensed_dist = squareform(dist)
        link = sch.linkage(condensed_dist, method='single')

        sorted_indices = self.get_quasi_diag(link)
        sorted_assets = returns.columns[sorted_indices].tolist()

        cov = returns[sorted_assets].cov()
        weights = pd.Series(1, index=sorted_assets)

        def recursive_bisection(indices, level=0):
            if len(indices) <= 1:
                return

            mid = len(indices) // 2
            left_indices = indices[:mid]
            right_indices = indices[mid:]

            left_var, left_weights = self.get_cluster_var(cov, left_indices)
            right_var, right_weights = self.get_cluster_var(cov, right_indices)

            alpha = 1 - (left_var / (left_var + right_var))

            for idx in left_indices:
                weights[sorted_assets[idx]] *= alpha
            for idx in right_indices:
                weights[sorted_assets[idx]] *= (1 - alpha)

            recursive_bisection(left_indices, level + 1)
            recursive_bisection(right_indices, level + 1)

        initial_indices = list(range(len(sorted_assets)))
        recursive_bisection(initial_indices)

        return weights

    def get_portfolio(self, n_stocks=5):
        base_portfolio = super().get_portfolio(n_stocks)

        long_tickers = base_portfolio['long']['Ticker'].tolist()
        returns = self.get_stock_returns(long_tickers)

        if returns.empty:
            base_portfolio['long']['HRP_Weight'] = 1.0 / len(long_tickers)
        else:
            hrp_weights = self.optimize_portfolio(returns)
            hrp_weights = hrp_weights / hrp_weights.sum()
            base_portfolio['long']['HRP_Weight'] = base_portfolio['long']['Ticker'].map(hrp_weights)

            # Calculate portfolio costs
            old_weights = getattr(self, '_previous_weights', pd.Series(index=base_portfolio['long']['Ticker'], data=0))
            new_weights = base_portfolio['long']['HRP_Weight']
            costs_analysis = self.calculate_portfolio_costs(old_weights, new_weights)
            base_portfolio['costs_analysis'] = costs_analysis

            # Calculate risk metrics
            risk_metrics = self.calculate_risk_metrics(returns)
            if risk_metrics:
                base_portfolio['risk_analysis'] = risk_metrics

                # Add risk-based suggestions
                if risk_metrics['risk_alerts']:
                    base_portfolio['risk_actions'] = [
                        alert['suggestion'] for alert in risk_metrics['risk_alerts']
                    ]

                    risk_contrib = pd.Series(risk_metrics['risk_contributions'])
                    high_risk_assets = risk_contrib[risk_contrib > risk_contrib.mean()]

                    if not high_risk_assets.empty:
                        base_portfolio['risk_actions'].append(
                            f'Consider reducing exposure to: {", ".join(high_risk_assets.index)}'
                        )

            # Store current weights for next rebalance
            self._previous_weights = new_weights

        return base_portfolio

def main():
    model = CombinedModelHRP()
    portfolio = model.get_portfolio(n_stocks=5)

    print("\nLong Portfolio (High Quality + Momentum) with HRP Weights:")
    print(portfolio['long'])

    if 'costs_analysis' in portfolio:
        print("\nPortfolio Costs Analysis:")
        print(portfolio['costs_analysis'])

    if 'risk_analysis' in portfolio:
        print("\nRisk Analysis:")
        print(portfolio['risk_analysis'])

    if 'risk_actions' in portfolio:
        print("\nRisk Management Suggestions:")
        for action in portfolio['risk_actions']:
            print(f"- {action}")

if __name__ == "__main__":
    main()

ERROR:yfinance:NBIS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:TTAN: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:NBIS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Error getting data for NBIS: single positional indexer is out-of-bounds


ERROR:yfinance:OS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']


Error getting data for OS: single positional indexer is out-of-bounds


ERROR:yfinance:TTAN: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']


Error getting data for TTAN: single positional indexer is out-of-bounds


/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)



Long Portfolio (High Quality + Momentum) with HRP Weights:
    Ticker                          Company      Sector  Combined_Score  \
194   SRRK        Scholar Rock Holding Corp  Healthcare        3.917468   
0      APP                    Applovin Corp  Technology        2.996741   
1       KC  Kingsoft Cloud Holdings Ltd ADR  Technology        2.665849   
140   GGAL     Grupo Financiero Galicia ADR   Financial        2.588490   
195    WGS             GeneDx Holdings Corp  Healthcare        2.020108   

     Quality_Score  XSMOM_Score  HRP_Weight  
194       0.787948     7.046987    0.012025  
0         0.947231     5.046251    0.205837  
1         0.551636     4.780063    0.110105  
140       1.190474     3.986507    0.569322  
195       0.947473     3.092743    0.102710  

Portfolio Costs Analysis:
{'turnover_ratio': 0.0, 'number_of_trades': 0, 'total_costs_usd': 0.0, 'costs_impact_bps': 0.0}

Risk Analysis:
{'VaR': 0.03579485499499908, 'CVaR': 0.04435814184916582, 'risk_alerts': [

<ipython-input-7-e4b962f2e4a4>:271: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.581346904715444' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights[sorted_assets[idx]] *= alpha


In [9]:
from finvizfinance.screener.overview import Overview
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
import time

class CombinedModel:
    def __init__(self):
        self.screener = Overview()
        self._previous_weights = None
        self.max_position = 0.35
        self.min_position = 0.05
        self.max_sector_exposure = 0.40

    def get_sector_data(self, sector):
        filters_dict = {
            'Exchange': 'NASDAQ',
            'Market Cap.': '+Mid (over $2bln)',
            'Average Volume': 'Over 500K',
            'Price': 'Over $10',
            'Sector': sector
        }
        self.screener.set_filter(filters_dict=filters_dict)
        return self.screener.screener_view()

    def calculate_xsmom(self, ticker, universe_returns):
        stock = yf.Ticker(ticker)
        hist = stock.history(period="6mo")
        if hist.empty:
            return 0

        stock_return = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
        mean_return = universe_returns['Returns'].mean()
        std_return = universe_returns['Returns'].std()

        return (stock_return - mean_return) / std_return

    def calculate_portfolio_costs(self, old_weights, new_weights, transaction_cost=2.0):
        weight_changes = abs(new_weights - old_weights)
        turnover = weight_changes.sum() / 2
        num_trades = len(weight_changes[weight_changes > 0.0001])
        total_costs = num_trades * transaction_cost
        portfolio_value = 100000
        costs_impact_bps = (total_costs / portfolio_value) * 10000

        return {
            'turnover_ratio': turnover,
            'number_of_trades': num_trades,
            'total_costs_usd': total_costs,
            'costs_impact_bps': costs_impact_bps
        }

    def calculate_risk_metrics(self, returns, confidence_level=0.95):
        if self._previous_weights is None:
            return None

        portfolio_returns = returns.dot(self._previous_weights)
        var = -np.percentile(portfolio_returns, (1 - confidence_level) * 100)
        cvar = -portfolio_returns[portfolio_returns <= -var].mean()

        var_limit = 0.02
        cvar_limit = 0.03

        alerts = []
        if var > var_limit:
            alerts.append({
                'level': 'WARNING',
                'message': f'VaR ({var:.2%}) exceeds limit of {var_limit:.2%}',
                'suggestion': 'Consider reducing position sizes or adding defensive stocks'
            })

        if cvar > cvar_limit:
            alerts.append({
                'level': 'CRITICAL',
                'message': f'CVaR ({cvar:.2%}) exceeds limit of {cvar_limit:.2%}',
                'suggestion': 'Immediate portfolio rebalancing recommended. Consider hedging positions'
            })

        marginal_var = {}
        for ticker in returns.columns:
            position_returns = returns[ticker]
            contribution = position_returns[portfolio_returns <= -var].mean() * self._previous_weights[ticker]
            marginal_var[ticker] = contribution

        return {
            'VaR': var,
            'CVaR': cvar,
            'risk_alerts': alerts,
            'risk_contributions': marginal_var
        }

    def sector_momentum_score(self, sector_returns):
        return (sector_returns - sector_returns.mean()) / sector_returns.std()

    def get_stock_fundamentals(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            hist = stock.history(period="1y")

            returns = hist['Close'].pct_change()
            growth_rate = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
            volatility = returns.std() * np.sqrt(252)

            return {
                'Ticker': ticker,
                'ROE': info.get('returnOnEquity', 0),
                'ROA': info.get('returnOnAssets', 0),
                'OperatingMargin': info.get('operatingMargins', 0),
                'Beta': info.get('beta', 1),
                'DebtToEquity': info.get('debtToEquity', 0),
                'GrowthRate': growth_rate,
                'Volatility': volatility
            }
        except Exception as e:
            print(f"Error getting data for {ticker}: {str(e)}")
            return None

    def analyze_sector(self, sector):
        df = self.get_sector_data(sector)
        if df.empty:
            return pd.DataFrame()

        universe_returns = []
        for ticker in df['Ticker']:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period="6mo")
                if not hist.empty:
                    returns = ((hist['Close'].iloc[-1] / hist['Close'].iloc[0]) - 1) * 100
                    universe_returns.append({
                        'Ticker': ticker,
                        'Returns': returns
                    })
                time.sleep(0.1)
            except:
                continue

        universe_returns_df = pd.DataFrame(universe_returns)

        detailed_data = []
        for ticker in df['Ticker']:
            data = self.get_stock_fundamentals(ticker)
            if data:
                xsmom = self.calculate_xsmom(ticker, universe_returns_df)
                data['XSMOM_Score'] = xsmom
                detailed_data.append(data)

        if not detailed_data:
            return pd.DataFrame()

        detailed_df = pd.DataFrame(detailed_data)
        df = df.merge(detailed_df, on='Ticker', how='inner')

        for col in ['ROE', 'ROA', 'OperatingMargin', 'Beta', 'DebtToEquity', 'GrowthRate', 'Volatility']:
            df[col] = df[col].clip(lower=df[col].quantile(0.05), upper=df[col].quantile(0.95))

        profitability = df[['ROE', 'ROA', 'OperatingMargin']].mean(axis=1)
        growth = df['GrowthRate']
        safety = -(df[['Beta', 'DebtToEquity', 'Volatility']].mean(axis=1))

        df['Profitability_Score'] = (profitability - profitability.mean()) / profitability.std()
        df['Growth_Score'] = (growth - growth.mean()) / growth.std()
        df['Safety_Score'] = (safety - safety.mean()) / safety.std()

        df['Quality_Score'] = (df['Profitability_Score'] + df['Growth_Score'] + df['Safety_Score']) / 3
        df['Combined_Score'] = (df['Quality_Score'] + df['XSMOM_Score']) / 2

        return df.sort_values('Combined_Score', ascending=False)

    def constrain_weights(self, weights, sectors=None):
        # Position limits
        weights = weights.clip(lower=self.min_position, upper=self.max_position)

        # Sector constraints if sectors provided
        if sectors is not None:
            sector_weights = weights.groupby(sectors).sum()
            overweight_sectors = sector_weights[sector_weights > self.max_sector_exposure]

            if not overweight_sectors.empty:
                scale_factor = self.max_sector_exposure / overweight_sectors
                for sector in overweight_sectors.index:
                    sector_mask = sectors == sector
                    weights[sector_mask] *= scale_factor[sector]

        # Renormalize
        return weights / weights.sum()

    def get_portfolio(self, n_stocks=5):
        sectors = ['Technology', 'Financial', 'Healthcare', 'Consumer Defensive']
        all_data = []

        for sector in sectors:
            sector_data = self.analyze_sector(sector)
            if not sector_data.empty:
                all_data.append(sector_data)

        if not all_data:
            raise ValueError("No data available for analysis")

        df = pd.concat(all_data, ignore_index=True)
        df = df.sort_values('Combined_Score', ascending=False)

        high_quality = df.head(n_stocks)
        low_quality = df.tail(n_stocks)

        if self._previous_weights is None:
            initial_weights = pd.Series(1.0/n_stocks, index=high_quality['Ticker'])
            self._previous_weights = self.constrain_weights(initial_weights, high_quality['Sector'])

        return {
            'long': high_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'short': low_quality[['Ticker', 'Company', 'Sector', 'Combined_Score', 'Quality_Score', 'XSMOM_Score']],
            'spread': high_quality['Combined_Score'].mean() - low_quality['Combined_Score'].mean()
        }

class CombinedModelHRP(CombinedModel):
    def get_stock_returns(self, tickers, period='1y'):
        returns_data = {}
        for ticker in tickers:
            try:
                stock = yf.Ticker(ticker)
                hist = stock.history(period=period)
                if not hist.empty:
                    returns = hist['Close'].pct_change().dropna()
                    returns_data[ticker] = returns
                time.sleep(0.1)
            except Exception as e:
                print(f"Error getting returns for {ticker}: {str(e)}")
        return pd.DataFrame(returns_data)

    def get_quasi_diag(self, link):
        num_items = link[-1, 3]
        cursors = [link[-1, 0], link[-1, 1]]
        sorted_items = []

        while len(sorted_items) < num_items:
            for cursor in cursors:
                if cursor < num_items:
                    sorted_items.append(int(cursor))
                    cursors.remove(cursor)
                else:
                    i = int(cursor - num_items)
                    cursors.remove(cursor)
                    cursors.extend([link[i, 0], link[i, 1]])

        return sorted_items

    def get_cluster_var(self, cov, cluster_indices):
        cluster_cov = cov.iloc[cluster_indices, cluster_indices]
        inv_diag = 1 / np.diag(cluster_cov)
        weights = inv_diag / inv_diag.sum()
        cluster_var = np.dot(np.dot(weights, cluster_cov), weights)
        return cluster_var, weights

    def optimize_portfolio(self, returns):
        if returns.empty:
            return pd.Series()

        corr = returns.corr()
        dist = np.sqrt(0.5 * (1 - corr))

        condensed_dist = squareform(dist)
        link = sch.linkage(condensed_dist, method='single')

        sorted_indices = self.get_quasi_diag(link)
        sorted_assets = returns.columns[sorted_indices].tolist()

        cov = returns[sorted_assets].cov()
        weights = pd.Series(1.0, index=sorted_assets)

        def recursive_bisection(indices, level=0):
            if len(indices) <= 1:
                return

            mid = len(indices) // 2
            left_indices = indices[:mid]
            right_indices = indices[mid:]

            left_var, left_weights = self.get_cluster_var(cov, left_indices)
            right_var, right_weights = self.get_cluster_var(cov, right_indices)

            alpha = 1 - (left_var / (left_var + right_var))

            for idx in left_indices:
                weights.loc[sorted_assets[idx]] *= alpha
            for idx in right_indices:
                weights.loc[sorted_assets[idx]] *= (1 - alpha)

            recursive_bisection(left_indices, level + 1)
            recursive_bisection(right_indices, level + 1)

        initial_indices = list(range(len(sorted_assets)))
        recursive_bisection(initial_indices)

        # Apply constraints
        weights = self.constrain_weights(weights)

        return weights

    def get_portfolio(self, n_stocks=5):
        base_portfolio = super().get_portfolio(n_stocks)

        long_tickers = base_portfolio['long']['Ticker'].tolist()
        returns = self.get_stock_returns(long_tickers)

        if returns.empty:
            weights = pd.Series(1.0 / len(long_tickers), index=long_tickers)
            base_portfolio['long']['HRP_Weight'] = self.constrain_weights(
                weights,
                base_portfolio['long']['Sector']
            )
        else:
            hrp_weights = self.optimize_portfolio(returns)
            hrp_weights = self.constrain_weights(
                hrp_weights,
                base_portfolio['long']['Sector']
            )
            base_portfolio['long']['HRP_Weight'] = base_portfolio['long']['Ticker'].map(hrp_weights)

            # Calculate portfolio costs
            old_weights = getattr(self, '_previous_weights', pd.Series(index=base_portfolio['long']['Ticker'], data=0))
            new_weights = base_portfolio['long']['HRP_Weight']
            costs_analysis = self.calculate_portfolio_costs(old_weights, new_weights)
            base_portfolio['costs_analysis'] = costs_analysis

            # Calculate risk metrics
            risk_metrics = self.calculate_risk_metrics(returns)
            if risk_metrics:
                base_portfolio['risk_analysis'] = risk_metrics

                # Add risk-based suggestions
                if risk_metrics['risk_alerts']:
                    base_portfolio['risk_actions'] = [
                        alert['suggestion'] for alert in risk_metrics['risk_alerts']
                    ]

                    risk_contrib = pd.Series(risk_metrics['risk_contributions'])
                    high_risk_assets = risk_contrib[risk_contrib > risk_contrib.mean()]

                    if not high_risk_assets.empty:
                        base_portfolio['risk_actions'].append(
                            f'Consider reducing exposure to: {", ".join(high_risk_assets.index)}'
                        )

            # Store current weights for next rebalance
            self._previous_weights = new_weights

        return base_portfolio

def main():
    model = CombinedModelHRP()
    portfolio = model.get_portfolio(n_stocks=5)

    print("\nLong Portfolio (High Quality + Momentum) with HRP Weights:")
    print(portfolio['long'])

    if 'costs_analysis' in portfolio:
        print("\nPortfolio Costs Analysis:")
        print(portfolio['costs_analysis'])

    if 'risk_analysis' in portfolio:
        print("\nRisk Analysis:")
        print(portfolio['risk_analysis'])

    if 'risk_actions' in portfolio:
        print("\nRisk Management Suggestions:")
        for action in portfolio['risk_actions']:
            print(f"- {action}")

if __name__ == "__main__":
    main()

ERROR:yfinance:NBIS: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:TTAN: Period '6mo' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']
ERROR:yfinance:NBIS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']


Error getting data for NBIS: single positional indexer is out-of-bounds


ERROR:yfinance:OS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']


Error getting data for OS: single positional indexer is out-of-bounds


ERROR:yfinance:TTAN: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', 'ytd', 'max']


Error getting data for TTAN: single positional indexer is out-of-bounds


/usr/local/lib/python3.10/dist-packages/finvizfinance/screener/base.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, pd.DataFrame(frame)], ignore_index=True)



Long Portfolio (High Quality + Momentum) with HRP Weights:
    Ticker                          Company      Sector  Combined_Score  \
194   SRRK        Scholar Rock Holding Corp  Healthcare        3.917468   
0      APP                    Applovin Corp  Technology        2.996741   
1       KC  Kingsoft Cloud Holdings Ltd ADR  Technology        2.665849   
140   GGAL     Grupo Financiero Galicia ADR   Financial        2.588490   
195    WGS             GeneDx Holdings Corp  Healthcare        2.020108   

     Quality_Score  XSMOM_Score  HRP_Weight  
194       0.787948     7.046987    0.066209  
0         0.947231     5.046251    0.272567  
1         0.551636     4.780063    0.145800  
140       1.190474     3.986507    0.379417  
195       0.947473     3.092743    0.136008  

Portfolio Costs Analysis:
{'turnover_ratio': 0.0, 'number_of_trades': 0, 'total_costs_usd': 0.0, 'costs_impact_bps': 0.0}

Risk Analysis:
{'VaR': 0.03579485499499908, 'CVaR': 0.04435813803386801, 'risk_alerts': [